In [1]:
import pandas
import networkx
import seaborn
import pyreadstat

In [2]:
df_original, meta_original = pyreadstat.read_sav("data-covid-source.sav")
print(f"There're {len(df_original.columns)} variables in the data:")
pandas.set_option('display.max_columns', None)
pandas.set_option("display.max_rows", None)
pandas.set_option("display.precision", 1)
display(df_original.describe(percentiles=[]).transpose())
pandas.reset_option("max_columns")
pandas.reset_option("max_rows")
pandas.reset_option("precision")

There're 510 variables in the data:


,count,mean,std,min,50%,max
Id,547.0,400.4,234.9,1.0,389.0,832.0
ATT_1_1,547.0,3.0,0.0,3.0,3.0,3.0
ATT_1_2,547.0,6.0,0.0,6.0,6.0,6.0
TIME_1,547.0,2026.2,3907.8,221.0,1274.0,55552.0
TIME_2,547.0,2363.2,5401.8,264.0,1408.0,88447.0
TIME_3,547.0,4137.7,15902.1,260.0,1556.0,278906.0
ZGODA3,547.0,1.0,0.0,1.0,1.0,1.0
ATT_2_1,547.0,4.0,0.0,4.0,4.0,4.0
ATT_2_2,547.0,5.0,0.0,5.0,5.0,5.0
ATT_2_3,547.0,4.0,0.0,4.0,4.0,4.0


In the above table there's some basic information on 510 variables measuring various psychological metrics from a longitude study of 547 people.<br>
There were 5 measurements, each one week apart from the others.<br>
The variables encode the number of measurement in their name by appending "\_t2", \_t3", ... to their name. For the first measurement "\_t1" is sometimes appended, sometimes not, depending on the variable.<br>
For some variables, not all 5 measurements are present - for example for communal narcissm ("cni" in the source data) the measurements are from week 1 and week 5 only. In the "cni" example, there are 16 variables from "cni1" to "cni16" describing various 16 metrics of communal narcissm, and are followed up with "cni1_t5" to "cni16_t5" (where "\_t5" indicates the week no. 5 of measurement taken).

The goal of this excercise is to:<br>
1. perform paired sample t test for following measurements;
1. compute [Cohen's d - effect size value](https://en.wikipedia.org/wiki/Effect_size#Cohen's_d);
1. plot Cohen's d change in time (with mean trajectories);
1. compute partial correlation networks and basic networks' metrics;
1. plot the networks (with only signifcant changes - uncorrected p>0.05);
1. plot correlogram for average absolute edge values between and within questionnaires;
1. plot basic graphs' metrics for particular measurements at different times to easily observe change (or lack of thereof);
1. compute and show in correlation matrix the Pearson's correlation of considered variables

Only selected variables are to be considered, and those are:
1. Knowledge ("wiedza");
1. Communal Narcissim ("cni");
1. Dissinfection ("odkazanie");
1. Stay-at-home ("siedz_w_domu");
1. Cleaning-hands ("mycie_rak");
1. Masks ("maseczki");
1. Threatened ("zagr_ja");
1. Series of WHO health metrics ("WHO_zdr", "WHO_sen", "WHO_samooc_ind", "WHO_zwiazki", "WHO_wsparcie");
